<a href="https://colab.research.google.com/github/amanglarus/image_classification_using-tensorflow/blob/main/classification_using_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/final_pipe_data.zip /content/

In [3]:
!unzip final_pipe_data.zip

Archive:  final_pipe_data.zip
   creating: final_pipe_data/
   creating: final_pipe_data/train/
   creating: final_pipe_data/train/3camera_blocked/
  inflating: final_pipe_data/train/3camera_blocked/2023-03-10-11-59-43-578167.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-10-11-59-46-938004.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-10-12-24-00-960970.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-10-13-16-12-293367.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-06-054966.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-06-990630.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-12-951789.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-14-708083.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-16-674829.jpg  
  inflating: final_pipe_data/train/3camera_blocked/2023-03-13-10-28-20-386384.jpg  
  inflating:

In [ ]:
!rm -r /content/final_pipe_data/test/3camera_blocked
# !rm -r /content/final_pipe_data/test/4nothing
!rm -r /content/final_pipe_data/train/3camera_blocked
# !rm -r /content/final_pipe_data/train/4nothing

In [ ]:
#!rm -r tshirt_laptop_data

## Convert all images to gray scale images

In [4]:
def rgb_to_gray(rgb_img): # provide image in rgb order not in bgr or any other order

  gamma = 1.04
  r_const, g_const, b_const = 0.2126, 0.7152, 0.0722

  r, g, b = rgb_img[:,:,0], rgb_img[:,:,1], rgb_img[:,:,2]

  gray_img = r_const*r**gamma + g_const*g**gamma + b_const*b**gamma

  return gray_img

In [5]:
grey_img = True

import glob
import cv2
dir_name = "/content/final_pipe_data"+"/*"
if grey_img:
    for folder_name in glob.glob(dir_name):
      for data_type_path in glob.glob(folder_name+"/*"):
          for img_path in glob.glob(data_type_path+"/*"):
              
              img = cv2.imread(img_path) #BGR color map
              rgb_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # BGR to RGB
              gray = rgb_to_gray(rgb_img)
              cv2.imwrite(img_path, gray)


# Install specific tensorflow version 

In [6]:
!pip install tensorflow-gpu==2.5.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 81.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4845 sha256=7b39b1eeffa14c458e8b7aed5537ebc4c7a3be7261fff90b388d53a4a19591c2
  Stored in directory: 

In [1]:
import tensorflow as tf
print(tf.__version__)


2.5.0


# Load data in pipe line to tarin the model

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [3]:
NUM_CLASSES = 4
BATCH_SIZE=32
IMAGE_SIZE = 96
NUM_CHANNEL = 1

IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE)
MODEL_IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNEL)

In [4]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir='/content/final_pipe_data/train'
val_data_dir='/content/final_pipe_data/test'

# from tensorflow.keras.layers.preprocessing.image_preprocessing import HORIZONTAL

train_datagen=ImageDataGenerator(#rescale=1./255, # 1./255 to scale
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')

val_datagen=ImageDataGenerator(#rescale=1./255
                               )




train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                  target_size=IMAGE_SHAPE,
                                                  batch_size=BATCH_SIZE,
                                                  color_mode="grayscale",
                                                  class_mode='sparse')

val_generator=val_datagen.flow_from_directory(val_data_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              color_mode="grayscale",
                                              class_mode='sparse')






Found 800 images belonging to 4 classes.
Found 110 images belonging to 4 classes.


In [5]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['1pipe_close', '2pipe_inside', '3camera_blocked', '4nothing']


# Build Model

### Mobilenet V3 from scratch


In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides, use_bias=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def depthwise_conv_block(inputs, strides, use_bias=True):
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

def bottleneck(inputs, filters, kernel_size, expansion_factor, strides, use_bias=True):
    in_channels = inputs.shape[-1]
    x = depthwise_conv_block(inputs, 1, use_bias)
    x = depthwise_conv_block(x, 1 use_bias)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv3_small(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)


    # # x3 = tf.keras.layers.Conv2D(1, 3, strides=1, padding='valid', use_bias=True)(inputs)
    # # x3 = tf.keras.layers.BatchNormalization()(x3)
    # # x3 = tf.keras.layers.ReLU()(x3)
    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(inputs)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(inputs)
    x = tf.keras.layers.Concatenate()([x1, x2])

    x = conv_block(x, 4, 3, strides=2)
    x = bottleneck(x, 6, 3, expansion_factor=1, strides=2)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=2)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = conv_block(x, 8, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(48)(x)
    x = tf.keras.layers.ReLU(6.)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv3_small(input_shape=MODEL_IMAGE_SHAPE, num_classes=NUM_CLASSES)
model.summary()


In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides, use_bias=True):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def depthwise_conv_block(inputs, strides, use_bias=True):3camera_blocked
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

def bottleneck(inputs, filters, kernel_size, expansion_factor, strides, use_bias=True):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, expansion_factor * in_channels, 1, 1, use_bias)
    x = conv_block(x, expansion_factor * in_channels, kernel_size, strides, use_bias)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv3_small(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)

    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(inputs)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(inputs)
    # # x3 = tf.keras.layers.Conv2D(1, 3, strides=1, padding='valid', use_bias=True)(inputs)
    # # x3 = tf.keras.layers.BatchNormalization()(x3)
    # # x3 = tf.keras.layers.ReLU()(x3)
    x = tf.keras.layers.Concatenate()([x1, x2])

    x = conv_block(x, 4, 3, strides=2)
    x = bottleneck(x, 6, 3, expansion_factor=1, strides=2)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    x = bottleneck(x, 6, 5, expansion_factor=1, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=2)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = bottleneck(x, 16, 5, expansion_factor=3, strides=1)
    # x = conv_block(x, 8, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(48)(x)
    x = tf.keras.layers.ReLU(6.)(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv3_small(input_shape=MODEL_IMAGE_SHAPE, num_classes=NUM_CLASSES)
model.summary()


Model: "model_18"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
average_pooling2d_13 (AveragePo (None, 31, 31, 1)    0           input_30[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_16 (MaxPooling2D) (None, 31, 31, 1)    0           input_30[0][0]                   
__________________________________________________________________________________________________
concatenate_4 (Concatenate)     (None, 31, 31, 2)    0           average_pooling2d_13[0][0]       
                                                                 max_pooling2d_16[0][0]    

### Mobilenet V2 from scratch

In [ ]:
import tensorflow as tf

def conv_block(inputs, filters, kernel_size, strides):
    x = tf.keras.layers.Conv2D(filters, kernel_size, strides=strides, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    return x

def bottleneck(inputs, filters, kernel_size, t, strides, use_bias=False):
    in_channels = inputs.shape[-1]
    x = conv_block(inputs, t * in_channels, 1, 1)
    x = tf.keras.layers.DepthwiseConv2D(kernel_size, strides=strides, padding='same', use_bias=use_bias)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Conv2D(filters, 1, strides=1, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    if strides == 1 and in_channels == filters:
        x = tf.keras.layers.Add()([x, inputs])
    return x

def mobilenetv2(input_shape, num_classes):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = conv_block(inputs, 32, 3, strides=2)
    x = bottleneck(x, 16, 3, t=1, strides=1)
    x = bottleneck(x, 24, 3, t=6, strides=2)
    x = bottleneck(x, 24, 3, t=6, strides=1)
    x = bottleneck(x, 32, 3, t=6, strides=2)
    x = bottleneck(x, 32, 3, t=6, strides=1)
    x = bottleneck(x, 64, 3, t=6, strides=2)
    x = bottleneck(x, 64, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 96, 3, t=6, strides=1)
    x = bottleneck(x, 160, 3, t=6, strides=2)
    x = bottleneck(x, 160, 3, t=6, strides=1)
    x = bottleneck(x, 320, 3, t=6, strides=1)
    x = conv_block(x, 1280, 1, strides=1)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, x)

# example usage
model = mobilenetv2(input_shape=(224, 224, 3), num_classes=10)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 112, 112, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 112, 112, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import MobileNetV3Large
#tf.keras.applications.MobileNetV3Small

base_model = MobileNetV3Large(
    input_shape=MODEL_IMAGE_SHAPE,
    alpha=0.4,
    include_top=False,
    weights=None, #  'imagenet'
    #input_tensor=None,
    #pooling=None,
    #classes=1000,
    #classifier_activation='softmax',
    #**kwargs
)

for layer in base_model.layers: #base_model.layers[:-6] means that consider layer only till 6th last layer from layer 1
    layer.trainable = True  



# penultimate_layer = model.layers[-2]
# new_top_layer = tf.keras.layers.Dense(1)(penultimate_layer.output)  # create new FC layer and connect it to the rest of the model
# new_model = tf.keras.models.Model(model.input, new_top_layer)



# Use a Sequential model to add a trainable classifier on top
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(NUM_CLASSES, activation="softmax"),
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
MobilenetV3large (Functional (None, 3, 3, 1280)        1031008   
_________________________________________________________________
global_average_pooling2d_17  (None, 1280)              0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 64)                81984     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 2)                 130       
Total params: 1,113,122
Trainable params: 1,102,962
Non-trainable params: 10,160
_______________________________________

##### MobileNetV2

In [ ]:
# https://github.com/Haikoitoh/paper-implementation/blob/main/MobileNetV2.ipynb

from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name = prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    x = ReLU(6,name = prefix +'expand_relu')(x)
    return x

def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    x = ReLU(6,name=prefix +'dw_relu')(x)
    return x

def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = Conv2D(filters = out_channels,kernel_size = 1,padding='same',use_bias=False,name= prefix + 'compress')(x)
    x = BatchNormalization(name=prefix +'compress_bn')(x)
    return x


def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y


def MobileNetV2(input_image = (224,224,3), n_classes=1000):
    input = Input(input_image)

    x1 = tf.keras.layers.AveragePooling2D(pool_size=(5, 5), strides=3)(input)
    x2 = tf.keras.layers.MaxPooling2D(pool_size=(5, 5), strides=3)(input)
    x = tf.keras.layers.Concatenate()([x1, x2])    

    x = Conv2D(1,kernel_size=3,strides=(2,2),padding = 'same', use_bias=False)(x)
    x = BatchNormalization(name='conv1_bn')(x)
    x = ReLU(6, name = 'conv1_relu')(x)

    # 17 Bottlenecks

    x = depthwise_block(x,stride=1,block_id=1)
    x = projection_block(x, out_channels=4,block_id=1)

    x = Bottleneck(x, t = 1, filters = x.shape[-1], out_channels = 6, stride = 2,block_id = 2)
    x = Bottleneck(x, t = 1, filters = x.shape[-1], out_channels = 6, stride = 1,block_id = 3)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)

    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
    # x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)

    # x = Bottleneck(x, t = 2, filters = x.shape[-1], out_channels = 8, stride = 1,block_id = 17)


    #1*1 conv
    x = Conv2D(filters = 6,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
    x = BatchNormalization(name='last_bn')(x)
    x = ReLU(6,name='last_relu')(x)
    
    #AvgPool 7*7
    x = GlobalAveragePooling2D(name='global_average_pool')(x)
    # x = tf.keras.layers.AveragePooling2D(pool_size=(6, 6), strides=3)(x)
    
    #Lstm Layer Added

    # x = tf.keras.layers.Reshape((1, 6))(x)
    # x = tf.keras.layers.LSTM(4, return_sequences=True)(x)

    #Lstm layer Ended

    x = tf.keras.layers.Dense(32, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)

    return model


# n_classes = 1000
# input_shape = (224,224,3)

# model = MobileNetV2(input_shape,n_classes)
# model.summary()   

model = MobileNetV2(MODEL_IMAGE_SHAPE, NUM_CLASSES)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 31, 31, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 31, 31, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 31, 31, 2)    0           average_pooling2d[0][0]          
                                                                 max_pooling2d[0][0]          

In [6]:
model = tf.keras.models.Sequential([
  # tf.keras.layers.Dense(64, activation='relu', input_shape=MODEL_IMAGE_SHAPE),
  tf.keras.layers.Input(shape=MODEL_IMAGE_SHAPE),
  tf.keras.layers.GlobalAveragePooling2D(name='global_average_pool'),
  # tf.keras.layers.Flatten(),
  # tf.keras.layers.Dense(64, activation='relu', input_shape=MODEL_IMAGE_SHAPE),

  # tf.tile can be used in place of broadcast_to when you want to deploy it on javascript use tile for microcontroller use broadcast
  # broadcast is more optimized but accuracy degrade in it
  tf.keras.layers.Lambda(lambda x: tf.broadcast_to(tf.constant([[0.50, 0.023, 0.00025, 0.15]]), (tf.shape(x)[0], 4))) ,
  # tf.keras.layers.Flatten()
  tf.keras.layers.Softmax()
  
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pool (GlobalA (None, 1)                 0         
_________________________________________________________________
lambda (Lambda)              (None, 4)                 0         
_________________________________________________________________
softmax (Softmax)            (None, 4)                 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [7]:
tf.nn.softmax([0.50, 0.023, 0.00025, 0.15])

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.3410626 , 0.21167797, 0.20691665, 0.24034275], dtype=float32)>

In [8]:
p = tf.ones((1,96, 96, 1))
y_pred = model.predict(p)
print(y_pred)

[[0.3410626  0.21167797 0.20691665 0.24034275]]


In [9]:
p_int8 = tf.cast(p, tf.int8)

# Train the model

In [10]:
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


checkpoint = ModelCheckpoint("image_classification_checkpoint.h5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=2,
                          verbose=1,
                          restore_best_weights=True)

callbacks=[checkpoint,earlystop]

model.compile(loss='SparseCategoricalCrossentropy',
                   optimizer=Adam(learning_rate=0.0015),
                   metrics=['sparse_categorical_accuracy'])

epochs=10

batch_size = 50

history = model.fit(train_generator,
                        #  steps_per_epoch=num_train_samples//batch_size,
                         epochs=epochs,
                        #  callbacks=callbacks,
                         validation_data=val_generator,
                        #  validation_steps=num_val_samples//batch_size
                         )

Epoch 1/10
25/25 [==============================] - 4s 135ms/step - loss: 1.4074 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.4134 - val_sparse_categorical_accuracy: 0.2364
Epoch 2/10
25/25 [==============================] - 2s 66ms/step - loss: 1.4074 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.4134 - val_sparse_categorical_accuracy: 0.2364
Epoch 3/10
25/25 [==============================] - 2s 64ms/step - loss: 1.4074 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.4134 - val_sparse_categorical_accuracy: 0.2364
Epoch 4/10
25/25 [==============================] - 2s 66ms/step - loss: 1.4074 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.4134 - val_sparse_categorical_accuracy: 0.2364
Epoch 5/10
25/25 [==============================] - 2s 68ms/step - loss: 1.4074 - sparse_categorical_accuracy: 0.2500 - val_loss: 1.4134 - val_sparse_categorical_accuracy: 0.2364
Epoch 6/10
25/25 [==============================] - 3s 105ms/step - loss: 1.4074 - sparse_categorical_ac

# Convert from tensorflow to tensorflow lite

In [11]:
datagen=ImageDataGenerator(#rescale=1./255,
                                 rotation_range=10,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
generator=datagen.flow_from_directory(val_data_dir,
                                      target_size=IMAGE_SHAPE,
                                      batch_size=BATCH_SIZE,
                                      color_mode="grayscale",
                                      class_mode='sparse')
def representative_data_gen():
  i = 0
  for image_batch, labels_batch in generator:
    i = i+1
    if i > 20:
      break;
    yield [image_batch]

Found 110 images belonging to 4 classes.


In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

print('\nSetting the optimization flags..')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

print('\nConverting...')
tflite_model = converter.convert()

open("person_detect_model_data.tflite", "wb").write(tflite_model)

print("Done Conversion")


Setting the optimization flags..

Converting...
Done Conversion


#Inference on tflite model converted from tensorflow

In [13]:
import numpy as np
interpreter = tf.lite.Interpreter(model_path="/content/person_detect_model_data.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
interpreter.set_tensor(input_details[0]['index'], p_int8) #p is input image 1, 96, 96, 1

interpreter.invoke()

# Get the output tensor
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)
print("class belong to = ", np.argmax(output_data, axis=-1))

[[-41 -74 -75 -67]]
class belong to =  [0]


In [14]:
# Converted from int8 to float value
output_scale, output_zero_point = output_details[0]['quantization']
output_data_float = (output_data - output_zero_point) * output_scale
output_data_postprocessed = np.argmax(output_data_float, axis=-1)
print("output_scale = ", output_scale, " output_zero_point = ", output_zero_point)
print(output_data_float)
print(output_data_postprocessed)

output_scale =  0.00390625  output_zero_point =  -128
[[0.33984375 0.2109375  0.20703125 0.23828125]]
[0]


In [15]:
(-41 - (-128))*0.00390625

0.33984375

# Convert from tensorflow to c array

In [15]:
!xxd -i ./person_detect_model_data.tflite > person_detect_model_data.cpp

In [16]:
#!cp /content/person_detect_model_data.cpp /content/person_detect_model_data_copy.cpp

In [17]:

search_text1 = "unsigned char __person_detect_model_data_tflite[]"


replace_text1 = str('#include "person_detect_model_data.h" \n\
#ifdef __has_attribute \n\
#define HAVE_ATTRIBUTE(x) __has_attribute(x) \n\
#else \n\
#define HAVE_ATTRIBUTE(x) 0 \n\
#endif \n\
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__)) \n\
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4))) \n\
#else \n\
#define DATA_ALIGN_ATTRIBUTE \n\
#endif \n\
const unsigned char g_person_detect_model_data[] DATA_ALIGN_ATTRIBUTE')


search_text2 = "unsigned int __person_detect_model_data_tflite_len"

replace_text2 = "const int g_person_detect_model_data_len"

with open(r'/content/person_detect_model_data.cpp', 'r') as file:

    data = file.read()

    data = data.replace(search_text1, replace_text1)
    data = data.replace(search_text2, replace_text2)
  # data = data.replace(search_text2, replace_text2)

with open(r'/content/person_detect_model_data.cpp', 'w') as file:
    file.write(data)
    file.close()



print("Text replaced")


Text replaced
